### Descriptor protocol

 Data-Descriptor is a class which implements any of \_\_get__ and ( \_\_set__ or \_\_delete__) methods

 1. Whenever a class level attribute is accessed using an instance (e.g. class_instance.attr) and the attribute (attr)
 is already an instance of a descriptor class , attr.\_\_get__(class_instance,class) is called
 and the return value of \_\_get__ is returned to the accessor.

 2. Whenever a class level attribute is assigned using an instance (e.g. class_instance.attr = new_value) and
  the attribute (attr) is already an instance of a descriptor class , attr.\_\_set__(class_instance,value) is called

 3. Whenever a class level attribute is deleted using an instance (e.g. del class_instance.attr) and the attribute (attr)
  is already an instance of a descriptor class , attr.\_\_delete__(class_instance) is called

### A sample descriptor:

In [1]:
class LoggingDescriptor:

    def __init__(self, property_name):
        print(f"Descriptor got created for property : {property_name}")
        self.property_name = property_name

    # Note : the instance is the instance of the class for which the attribute is being set
    # owner is the class : useful in inheritance
    def __get__(self, instance, owner):
        print(f"Somebody asked for the value for {self.property_name} using class {owner}")
        if not instance:
            print("But didn't provide an instance!")
            return None
        return instance.__dict__.get(self.property_name)

    def __set__(self, instance, value):
        print(f"Somebody is changing the value for {self.property_name} with {value}")
        instance.__dict__[self.property_name] = value

    def __delete__(self, instance):
        print(f"Somebody is deleting value for {self.property_name}")
        del instance.__dict__[self.property_name]

In [2]:
class Person:
    # Note the descriptors are always assigned to class level attributes, but come into action when those attributes
    # are accessed/assigned/deleted using instance of the class (OR accessed through class name)
    # Also note that : We have created a separate descriptor for each attribute
    name = LoggingDescriptor("name")
    age = LoggingDescriptor("age")

    def __init__(self, name, age):
        # descriptors __set__ will be called with instance=self and value={name}/{age}
        self.name = name
        self.age = age

Descriptor got created for property : name
Descriptor got created for property : age


In [3]:
ajay = Person("Ajay", 45)
vijay = Person("Vijay", 23)

Somebody is changing the value for name with Ajay
Somebody is changing the value for age with 45
Somebody is changing the value for name with Vijay
Somebody is changing the value for age with 23


In [4]:
# calls the descriptors __get__
print(ajay.name)
# calls the descriptors __set__(ajay,53)
ajay.age = 53
# calls the descriptors __get__(ajay,Person)
print(ajay.name, ajay.age)
# calls the descriptors __delete__(ajay)
del ajay.name, ajay.age

Somebody asked for the value for name using class <class '__main__.Person'>
Ajay
Somebody is changing the value for age with 53
Somebody asked for the value for name using class <class '__main__.Person'>
Somebody asked for the value for age using class <class '__main__.Person'>
Ajay 53
Somebody is deleting value for name
Somebody is deleting value for age


In [5]:
# different person will have different values for properties (as we store the values in instance.__dict__)
print(vijay.name, vijay.age)

Somebody asked for the value for name using class <class '__main__.Person'>
Somebody asked for the value for age using class <class '__main__.Person'>
Vijay 23


### Using descriptors for encapsulations (what we actually do in getter/setter in java)

In [7]:
class AgeValidationDescriptor:
    property_name = 'age'

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.property_name)

    def __set__(self, instance, age):
        if age < 0 or age > 150:
            raise ValueError('Age should be between 0 to 150')
        instance.__dict__[self.property_name] = age


In [8]:
class NameDescriptor:
    property_name = 'name'

    def __get__(self, instance, owner):
        return instance.__dict__.get(self.property_name)

    def __set__(self, instance, name):
        if self.property_name in instance.__dict__:
            raise ValueError("Name already set")
        instance.__dict__[self.property_name] = name

In [9]:
class ValidPerson:
    name = NameDescriptor()
    age = AgeValidationDescriptor()

    def __str__(self):
        return f"{self.name}({self.age})"

In [10]:
pradeep = ValidPerson()

pradeep.name = 'Pradeep'
pradeep.age = 23

print("Valid person: ", pradeep)

Valid person:  Pradeep(23)


#### Now let's break the rules

In [11]:
try:
    print("Trying to reset the name of the person:")
    pradeep.name = 'Akshay'
except ValueError as v:
    print(v)

Trying to reset the name of the person:
Name already set


In [12]:
try:
    print("Trying to set a negative age:")
    pradeep.age = -45
except ValueError as v:
    print(v)

Trying to set a negative age:
Age should be between 0 to 150


In [13]:
print("Person still valid: ", pradeep)

Person still valid:  Pradeep(23)


##### Above code style looks good for generic descriptors like logging, timing etc, but not suitable for specific encapsulation, it looks overkill to create a new descriptor for each attribute

### Using descriptors like java style

In [14]:
# TODO